In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [6]:
pip install neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the CSV data into a DataFrame
csv_file_path = '/content/orders.csv'
df = pd.read_csv(csv_file_path).dropna()

# Print the columns and the first few rows of the DataFrame to verify
print("Columns in the DataFrame:", df.columns)
print("First few rows of the DataFrame:", df.head())


Columns in the DataFrame: Index(['orderid', 'customerid', 'employeeid', 'orderdate', 'requireddate',
       'shippeddate', 'shipvia', 'freight', 'shipname', 'shipcity',
       'shippostalcode', 'shipcountry'],
      dtype='object')
First few rows of the DataFrame:    orderid customerid  employeeid   orderdate requireddate shippeddate  \
0    10248      VINET           5  04/07/1996   01/08/1996  16/07/1996   
1    10249      TOMSP           6  05/07/1996   16/08/1996  10/07/1996   
2    10250      HANAR           4  08/07/1996   05/08/1996  12/07/1996   
3    10251      VICTE           3  08/07/1996   05/08/1996  15/07/1996   
4    10252      SUPRD           4  09/07/1996   06/08/1996  11/07/1996   

   shipvia  freight                   shipname        shipcity shippostalcode  \
0        3    32.38  Vins et alcools Chevalier           Reims          51100   
1        1    11.61         Toms Spezialitäten         Münster          44087   
2        2    65.83              Hanari Carnes 

In [ ]:
from neo4j import GraphDatabase

# Initialize Neo4j driver
uri = "neo4j+s://bd0c01e9.databases.neo4j.io"
user = "neo4j"
password = "96WS8x-pw6ivgke550VDhg4evtuVcTy4qhLjVMQPx5U"
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_nodes_from_csv(driver, csv_file_path):
    # Load CSV into DataFrame
    df = pd.read_csv(csv_file_path).dropna()

    with driver.session() as session:
        for index, row in df.iterrows():
            session.run("""
                CREATE (o:Order {
                    orderid: $orderid,
                    customerid: $customerid,
                    shipname: $shipname,
                    shipcity: $shipcity,
                    shipcountry: $shipcountry
                })
                """,
                orderid=row['orderid'],
                customerid=row['customerid'],
                shipname=row['shipname'],
                shipcity=row['shipcity'],
                shipcountry=row['shipcountry']
            )
    print("Nodes created successfully.")

# Call the function to create nodes from the CSV
create_nodes_from_csv(driver, csv_file_path)


Nodes created successfully.


In [ ]:
def query_neo4j(driver, query):
    with driver.session() as session:
        result = session.run(query)
        records = result.data()
    return records

# Example query to count orders by shipping company
query = """
MATCH (o:Order)
RETURN o.shipname AS shipname, COUNT(o) AS order_count
ORDER BY order_count DESC
"""
results = query_neo4j(driver, query)

# Display results
for record in results:
    print(f"Shipping Company: {record['shipname']}, Order Count: {record['order_count']}")


Shipping Company: Save-a-lot Markets, Order Count: 62
Shipping Company: Ernst Handel, Order Count: 56
Shipping Company: QUICK-Stop, Order Count: 56
Shipping Company: Folk och fä HB, Order Count: 38
Shipping Company: HILARION-Abastos, Order Count: 36
Shipping Company: Berglunds snabbköp, Order Count: 36
Shipping Company: Rattlesnake Canyon Grocery, Order Count: 34
Shipping Company: Bon app', Order Count: 32
Shipping Company: Wartian Herkku, Order Count: 30
Shipping Company: Frankenversand, Order Count: 30
Shipping Company: Hanari Carnes, Order Count: 28
Shipping Company: White Clover Markets, Order Count: 28
Shipping Company: Lehmanns Marktstand, Order Count: 28
Shipping Company: Königlich Essen, Order Count: 28
Shipping Company: Mère Paillarde, Order Count: 26
Shipping Company: La maison d'Asie, Order Count: 26
Shipping Company: Around the Horn, Order Count: 26
Shipping Company: Bottom-Dollar Markets, Order Count: 26
Shipping Company: Suprêmes délices, Order Count: 24
Shipping Company:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

model_name_or_id = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

def generate_answer(question, context):
    prompt = f'''
    Answer the question based on the provided context:

    {question},

    context: {context}
    '''

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generation_config = GenerationConfig(
        do_sample=True,
        top_k=1,
        temperature=0.99,
        max_new_tokens=900,
        pad_token_id=tokenizer.eos_token_id
    )
    outputs = model.generate(**inputs, generation_config=generation_config)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# Example usage
question = "How many orders were shipped by each shipping company?"
context = " ".join([f"{record['shipname']}: {record['order_count']} orders" for record in results])
answer = generate_answer(question, context)
print(answer)



    Answer the question based on the provided context:

    How many orders were shipped by each shipping company?,

    context: Save-a-lot Markets: 62 orders Ernst Handel: 56 orders QUICK-Stop: 56 orders Folk och fä HB: 38 orders HILARION-Abastos: 36 orders Berglunds snabbköp: 36 orders Rattlesnake Canyon Grocery: 34 orders Bon app': 32 orders Wartian Herkku: 30 orders Frankenversand: 30 orders Hanari Carnes: 28 orders White Clover Markets: 28 orders Lehmanns Marktstand: 28 orders Königlich Essen: 28 orders Mère Paillarde: 26 orders La maison d'Asie: 26 orders Around the Horn: 26 orders Bottom-Dollar Markets: 26 orders Suprêmes délices: 24 orders LILA-Supermercado: 24 orders Queen Cozinha: 24 orders Blondel père et fils: 22 orders Reggiani Caseifici: 22 orders Vaffeljernet: 22 orders LINO-Delicateses: 22 orders Victuailles en stock: 20 orders Ottilies Käseladen: 20 orders Magazzini Alimentari Riuniti: 20 orders Tortuga Restaurante: 20 orders Ricardo Adocicados: 20 orders B's Beverag

**COMBINED NEO4J AND VECTORDB**

In [3]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existin

In [4]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 13.8 MB/s eta 0:00:00


In [5]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00


In [8]:
import faiss
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import time


paper_text_df = pd.read_csv('/content/orders.csv').dropna()
print("Columns in the DataFrame:", paper_text_df.columns)
print("First few rows of the DataFrame:", paper_text_df.head())

paper_text = (paper_text_df['orderid'].astype(str) + ' ' +
              paper_text_df['customerid'] + ' ' +
              paper_text_df['shipname'] + ' ' +
              paper_text_df['shipcity'] + ' ' +
              paper_text_df['shipcountry']).tolist()

# Check GPU availability
if not torch.cuda.is_available():
    print("Warning: No GPU detected. Processing will be slow. Please add a GPU to this notebook")

# Initialize SentenceTransformer model
model_name = 'LABSE'
model1 = SentenceTransformer(model_name)

# Encode the corpus
print("Encoding the corpus. This might take a while")
corpus_embeddings = model1.encode(paper_text, show_progress_bar=True, convert_to_tensor=True)

# Convert embeddings to numpy array for FAISS
corpus_embeddings_np = corpus_embeddings.cpu().numpy()

# Create and populate FAISS index
dimension = corpus_embeddings_np.shape[1]  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # Using L2 distance (Euclidean distance)
index.add(corpus_embeddings_np)  # Add embeddings to the index

def search_faiss(inp_question, num_res):
    res = {}
    start_time = time.time()
    question_embedding = model1.encode(inp_question, convert_to_tensor=True).cpu().numpy()
    # Reshape the question embedding to a 2D array
    question_embedding = question_embedding.reshape(1, -1)  # Reshape to (1, dimension)
    distances, indices = index.search(question_embedding, num_res)  # Search using FAISS
    end_time = time.time()

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time - start_time))

    for idx in indices[0]:
        res[idx] = paper_text[idx]

    df = pd.DataFrame(list(res.items()), columns=['id', 'res'])
    return df


Columns in the DataFrame: Index(['orderid', 'customerid', 'employeeid', 'orderdate', 'requireddate',
       'shippeddate', 'shipvia', 'freight', 'shipname', 'shipcity',
       'shippostalcode', 'shipcountry'],
      dtype='object')
First few rows of the DataFrame:    orderid customerid  employeeid   orderdate requireddate shippeddate  \
0    10248      VINET           5  04/07/1996   01/08/1996  16/07/1996   
1    10249      TOMSP           6  05/07/1996   16/08/1996  10/07/1996   
2    10250      HANAR           4  08/07/1996   05/08/1996  12/07/1996   
3    10251      VICTE           3  08/07/1996   05/08/1996  15/07/1996   
4    10252      SUPRD           4  09/07/1996   06/08/1996  11/07/1996   

   shipvia  freight                   shipname        shipcity shippostalcode  \
0        3    32.38  Vins et alcools Chevalier           Reims          51100   
1        1    11.61         Toms Spezialitäten         Münster          44087   
2        2    65.83              Hanari Carnes 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

...

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(input_path, "pytorch_

Encoding the corpus. This might take a while


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [9]:
from neo4j import GraphDatabase

# Initialize Neo4j driver
uri = "neo4j+s://bd0c01e9.databases.neo4j.io"
user = "neo4j"
password = "96WS8x-pw6ivgke550VDhg4evtuVcTy4qhLjVMQPx5U"
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_nodes_from_csv(driver, csv_file_path):
    df = pd.read_csv(csv_file_path).dropna()
    with driver.session() as session:
        for index, row in df.iterrows():
            session.run("""
                CREATE (o:Order {
                    orderid: $orderid,
                    customerid: $customerid,
                    shipname: $shipname,
                    shipcity: $shipcity,
                    shipcountry: $shipcountry
                })
                """,
                orderid=row['orderid'],
                customerid=row['customerid'],
                shipname=row['shipname'],
                shipcity=row['shipcity'],
                shipcountry=row['shipcountry']
            )
    print("Nodes created successfully.")

# Call the function to create nodes from the CSV
create_nodes_from_csv(driver, '/content/orders.csv')

def query_neo4j(driver, query):
    with driver.session() as session:
        result = session.run(query)
        records = result.data()
    return records

# Example query to count orders by shipping company
query = """
MATCH (o:Order)
RETURN o.shipname AS shipname, COUNT(o) AS order_count
ORDER BY order_count DESC
"""
results = query_neo4j(driver, query)

# Display results
for record in results:
    print(f"Shipping Company: {record['shipname']}, Order Count: {record['order_count']}")


Nodes created successfully.
Shipping Company: Save-a-lot Markets, Order Count: 124
Shipping Company: Ernst Handel, Order Count: 112
Shipping Company: QUICK-Stop, Order Count: 112
Shipping Company: Folk och fä HB, Order Count: 76
Shipping Company: HILARION-Abastos, Order Count: 72
Shipping Company: Berglunds snabbköp, Order Count: 72
Shipping Company: Rattlesnake Canyon Grocery, Order Count: 68
Shipping Company: Bon app', Order Count: 64
Shipping Company: Wartian Herkku, Order Count: 60
Shipping Company: Frankenversand, Order Count: 60
Shipping Company: Hanari Carnes, Order Count: 56
Shipping Company: White Clover Markets, Order Count: 56
Shipping Company: Lehmanns Marktstand, Order Count: 56
Shipping Company: Königlich Essen, Order Count: 56
Shipping Company: Mère Paillarde, Order Count: 52
Shipping Company: La maison d'Asie, Order Count: 52
Shipping Company: Around the Horn, Order Count: 52
Shipping Company: Bottom-Dollar Markets, Order Count: 52
Shipping Company: Suprêmes délices, Or

In [10]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

# Load the model and tokenizer
model_name_or_id = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

def generate_answer(question, context):
    prompt = f'''
    Answer the question based on the provided context:

    {question},

    context: {context}
    '''

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generation_config = GenerationConfig(
        do_sample=True,
        top_k=1,
        temperature=0.99,
        max_new_tokens=900,
        pad_token_id=tokenizer.eos_token_id
    )
    outputs = model.generate(**inputs, generation_config=generation_config)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

def demo_function(question):
    # FAISS search
    faiss_results = search_faiss(question, 4)
    faiss_context = " - ".join(faiss_results['res'])
    faiss_answer = generate_answer(question, faiss_context)

    # Neo4j search
    neo4j_query = """
    MATCH (o:Order)
    RETURN o.shipname AS shipname, COUNT(o) AS order_count
    ORDER BY order_count DESC
    """
    neo4j_results = query_neo4j(driver, neo4j_query)
    neo4j_context = " - ".join([f"{record['shipname']}: {record['order_count']} orders" for record in neo4j_results])
    neo4j_answer = generate_answer(question, neo4j_context)

    return {"FAISS Answer": faiss_answer, "Neo4j Answer": neo4j_answer}



config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
iface = gr.Interface(
    fn=demo_function,
    inputs=gr.Textbox(label="Ask a Question"),
    outputs=gr.JSON(label="Results")
)


iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://60326fc2aecac6984a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


FileNotFoundError: Cannot find file: /content/model.pt